# LAB 08 - 메일링 리스트 / 데이터베이스 연동
## 1. 메일 발송 모듈 제작

In [1]:
import os
from smtplib import SMTP
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication
from email.mime.multipart import MIMEMultipart

def sendMail(from_addr, to_addr, subject, content, files=[]):
    content_type='plain'
    username='cody634455@gmail.com'
    password='dwse mkxb bhvh jtcj'
    smtp='smtp.gmail.com'
    port=587
    msg=MIMEMultipart()
    msg['Subject']=subject
    msg['From']=from_addr
    msg['To']=to_addr
    msg.attach(MIMEText(content, content_type))
    if files:
        for file_item in files:
            if os.path.exists(file_item):
                with open(file_item, 'rb') as f:
                    basename=os.path.basename(file_item)
                    part=MIMEApplication(f.read(),Name=basename)
                    part['Content-Disposition']='attachment; filename="%s"' %basename
                    msg.attach(part)
                    print(basename,'(이)가 첨부되었습니다.')
    mail=SMTP(smtp)
    mail.ehlo()
    mail.starttls()
    mail.login(username,password)
    mail.sendmail(from_addr,to_addr,msg.as_string())
    mail.quit()

if __name__=="__main__":
    sendMail('cody634455@gmail.com','cody6344@naver.com','메일 발송 모듈 테스트','이것은 테스트 입니다.')

In [6]:
from mylibrary import MyMailer
import datetime as dt

today=dt.datetime.now()
year=today.year
month=today.month
day=today.day
print(year, month, day)

2025 10 27


In [13]:
fromAddr='운영지원팀 <cody634455@gmail.com>'
subjectTmpl='{name}님의 {yy}년 {mm}월 급여명세서 입니다.'
with open('mail/content.txt','r',encoding='utf-8') as f:
    contentTmpl=f.read()
    print(contentTmpl)

안녕하세요 {name}님

{yy}년도 {mm}월 급여명세서와 결산보고서 보내드립니다.

귀하의 노고에 감사드립니다.

- {yy}년 {mm}월 {dd}일 / 운영지원팀 드림


In [15]:
with open('mail/mail_list.csv','r',encoding='utf-8') as f:
    csv=f.readlines()
    for line in csv:
        name,email,file1,file2=line.strip().split(',')
        #print(name,email,file1,file2)
        toAddr='{name} <{email}>'.format(name=name, email=email)
        #print(toAddr)
        subject=subjectTmpl.format(name=name,yy=year,mm=month)
        #print(subject)
        content=contentTmpl.format(name=name,yy=year,mm=month,dd=day)
        #print(content)
        MyMailer.sendMail(fromAddr,toAddr,subject,content,[file1,file2])

document.pptx (이)가 첨부되었습니다.
pay1.xlsx (이)가 첨부되었습니다.
document.pptx (이)가 첨부되었습니다.
pay2.xlsx (이)가 첨부되었습니다.
document.pptx (이)가 첨부되었습니다.
pay3.xlsx (이)가 첨부되었습니다.
document.pptx (이)가 첨부되었습니다.
pay4.xlsx (이)가 첨부되었습니다.
document.pptx (이)가 첨부되었습니다.
pay5.xlsx (이)가 첨부되었습니다.
